In [3]:
!pip install praw

In [6]:
import requests
import requests.auth
import pandas as pd
from newspaper import Article 
import praw
import json
with open('config.json') as f:
    data = json.load(f)
reddit_cred = data['Reddit']
watson_cred = data['Watson']
newspaper_cred = data['News']
sql_cred = data['SQL']


## skip the following unless you wanna scrape new stuff

In [5]:

reddit = praw.Reddit(**reddit_cred)

import nltk

import re
# nltk.download()
from praw.models import MoreComments
comment_dict = { "post_id":[], 
                  'post_title':[],

                "id": [],
                        "author":[], 
                        "body":[],
                        "created": [],
                         'score':[],
                 'is_submitter':[],
                  'parent_id':[]}
topics_dict = { "title":[], 
                "score":[], 
                "id":[], "url":[], 
                "comms_num": [], 
                "created": [], 
                "body":[],
                "image":[],
                "keywords":[],
                "summary":[]
                  }
for submission in reddit.subreddit('news').controversial(time_filter = 'month', limit=300):
    submission.comment_sort = 'top'
#     print(submission.url)
    article = Article(submission.url)
    try:
        article.download()
        article.parse()

        article.nlp()
    except:
        continue
    topics_dict["title"].append(re.sub(r'[^\x00-\x7F]', '', submission.title.replace('"', "'")))
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(re.sub(r'[^\x00-\x7F]', '',article.text.replace('"', "'")))
    topics_dict["image"].append(article.top_image)
    topics_dict["keywords"].append(', '.join(article.keywords).replace('"', "'"))
    topics_dict["summary"].append(re.sub(r'[^\x00-\x7F]', '', article.summary.replace('"', "'")))
    
    try:
        submission.comments.replace_more(limit=None)
    except AssertionError:
        continue
    for top_level_comment in submission.comments.list():
#         print(top_level_comment.parent.id)
#         print(len(submission.comments.list()))
        comment_dict['post_id'].append(submission.id)
        comment_dict['is_submitter'].append(top_level_comment.is_submitter)
        comment_dict['id'].append(top_level_comment.id)
        comment_dict['author'].append(top_level_comment.author)
        comment_dict['body'].append(re.sub(r'[^\x00-\x7F]', '', top_level_comment.body))
        comment_dict['score'].append(top_level_comment.score)
        comment_dict['created'].append(top_level_comment.created_utc)
        comment_dict['parent_id'].append(top_level_comment.parent_id)
        comment_dict['post_title'].append(submission.title)

#         for second_level_comment in top_level_comment.replies:
#             comment_dict['post_id'].append(submission.id)
#             comment_dict['id'].append(second_level_comment.id)
#             comment_dict['body'].append(re.sub(r'[^\x00-\x7F]', '',second_level_comment.body))
#             comment_dict['score'].append(second_level_comment.score)
#             comment_dict['is_submitter'].append(second_level_comment.is_submitter)
#             comment_dict['parent_id'].append(top_level_comment.id)
#             comment_dict['created'].append(second_level_comment.created_utc)
#             comment_dict['post_title'].append(submission.title)
#             try:
#                 comment_dict['author'].append(second_level_comment.author.name)
#             except:
#                 comment_dict['author'].append('None')
        print(len(comment_dict['post_id']))

        if len(comment_dict['post_id']) == 29000:
        
            comment_data = pd.DataFrame(comment_dict)
            topics_data = pd.DataFrame(topics_dict)
            


In [7]:
'Hundreds mourn 22-year-old "disabled" Palestinian shot dead by Israeli army'.replace('"', '\'')

"Hundreds mourn 22-year-old 'disabled' Palestinian shot dead by Israeli army"

## Start here, reading articles and convertingtime

In [25]:
from pandas import Series, DataFrame, Panel
import numpy as np

In [8]:
from sqlalchemy import create_engine
conn_string = 'mysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = sql_cred["host"], 
    user = sql_cred["user"],
    password = sql_cred["password"], 
    db = 'MemeNews')
engine = create_engine(conn_string)

q = '''select * from every_article'''

In [17]:
df = pd.read_sql(q, con = engine)


In [35]:
df['created'] = pd.to_datetime(df['created'], unit='s')
df.set_index('created')

,index,title,score,id,url,comms_num,body,image,keywords,summary
created,,,,,,,,,,
2018-11-06 18:06:40,0,4 military officers indicted for surveilling f...,32,9un4go,http://english.yonhapnews.co.kr/news/2018/11/0...,4,'The DSC formed and operated a task force to s...,https://img6.yna.co.kr/photo/cms/2018/07/24/07...,"families, situation, military, ferry, unfavora...",'The DSC formed and operated a task force to s...
2018-11-11 00:33:37,1,'They Didn't Want a Flag': Protesters Vandaliz...,18,9vvn4o,http://insider.foxnews.com/2018/11/10/proteste...,121,"The owner of a pizzeria in New Paltz, New York...",http://insider.foxnews.com/sites/insider.foxne...,"didnt, political, pizza, officer, lisante, fla...","The owner of a pizzeria in New Paltz, New York..."
2018-11-09 18:29:42,2,US navy ship ignored sinking migrants' cries f...,75,9vjdck,https://www.theguardian.com/world/2018/nov/09/...,144,US navy ship ignored sinking migrants' cries f...,https://i.guim.co.uk/img/media/88dc3f38b302149...,"boat, american, ship, navy, migrants, saw, sin...",US navy ship ignored sinking migrants' cries f...
2018-11-06 16:32:16,3,Scientists say mysterious 'Oumuamua' object co...,0,9umo6j,https://www.nbcnews.com/mach/science/scientist...,52,Get the Mach newsletter.\n\nBy David Freeman\n...,https://media2.s-nbcnews.com/j/newscms/2018_45...,"way, mysterious, scientists, alien, lightsail,...",Scientists have been puzzling over Oumuamua ev...
2018-11-27 00:46:50,4,Suspected Illegal Immigrant Charged in Hit-and...,38,a0kzti,http://insider.foxnews.com/2018/11/26/illegal-...,137,A suspected illegal immigrant is facing charge...,http://insider.foxnews.com/sites/insider.foxne...,"president, immigrant, tx, death, father, suspe...",A suspected illegal immigrant is facing charge...
2018-11-12 01:15:51,5,"Migrant caravan leaves Mexico City, with eyes ...",0,9w5hsv,https://www.foxnews.com/world/migrant-caravan-...,138,The caravan of several thousand Central Americ...,https://media2.foxnews.com/BrightCove/69494009...,"told, border, trump, city, honduras, eyes, car...","The caravan, which now is made up of about 4,0..."
2018-11-15 09:43:24,6,Black student has police called on her by prof...,12,9x6jd4,https://www.nbcnews.com/news/amp/ncna936256,172,"Nov. 14, 2018 / 2:20 PM ET / Updated Nov. 15, ...",https://media3.s-nbcnews.com/i/MSNBC/Component...,"investigation, called, students, classroom, cl...","Apurva Rawal, a student in the class, told NBC..."
2018-11-12 06:19:37,7,PressTV-Israeli settlers block entry of goods ...,4,9w8329,https://www.presstv.com/Detail/2018/11/11/5797...,51,Extremist Israeli settlers have blocked a road...,https://cdn.presstv.com/photo/20181111/2e92013...,"crossing, palestinians, supplies, block, goods...",Extremist Israeli settlers have blocked a road...
2018-11-28 04:34:38,8,Mexico asks U.S. to investigate use of tear ga...,27,a0yqgr,https://www.reuters.com/article/us-usa-immigra...,201,"Migrants, part of a caravan of thousands from ...",https://s3.reutersmedia.net/resources/r/?m=02&...,"near, asks, border, tear, tijuana, investigate...","Migrants, part of a caravan of thousands from ..."


In [36]:
df.groupby('created').resample('W-Mon', on='created').sum().reset_index().sort_values(by='created')


KeyError: 'The grouper name created is not found'

In [30]:
ndf = df.pivot_table(index=['created'], 
                    columns='title',
                    aggfunc='size')
ndf

title,'Baby It's Cold Outside' cut from Cleveland radio station after #MeToo,'Everything is a target': Gaza residents decry Israeli strikes,'I Cannot Enforce This Law': Police Chief Pushes Back on Washington's New Gun Restrictions,"'I'm ready,' Mexican citizen says before being executed in Texas for killing of wife, kids",'They Didn't Want a Flag': Protesters Vandalize Patriotic Mural Painted on NY Pizza Shop,'World's oldest rebel' war veteran dies,"'You're a terrorist,' says note sent to 10-year-old Muslim schoolgirl in Massachusetts","1 shot, 1 injured in road rage shooting while in line for Zoo Lights","2 California teens killed execution-style in triple slaying in Mexico, reports say","2,550 US citizens applied for asylum in Canada in 2017. That's more than 6 times as many as in 2016",...,"White House backs down from legal fight, restores Jim Acosta's press pass",Woman Who Popped Open Some Pringles Before Paying Will Be Jailed the Day the Fun Stops,Woman booked into Miami mens jail after she was wrongly deemed transgender has lawsuit upheld,"Woman goes on racist, homophobic rant on plane, reportedly threatens a flight attendant","Woman in alleged GoFundMe scam claims she was set up by boyfriend, homeless man",Woman shot while sitting on porch in St. Pete - Story,"Woman, puppy attacked by pit bulls in Indian Trail, NC",Wonder Woman stuntwoman O'Neil dies,Yaxley dog attack leaves baby in 'critical' condition,Yemen conflict: US defends backing for Saudi-led coalition
created,,,,,,,,,,,,,,,,,,,,,
2018-11-05 23:11:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-06 01:24:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-06 02:57:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-06 03:18:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-06 03:20:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-06 06:49:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-06 08:58:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-06 11:22:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-06 16:32:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
from matplotlib import pyplot
df.index
# df.groupby(df.date).count()

RangeIndex(start=0, stop=277, step=1)

In [ ]:

a = pd.to_datetime(pd.Series(a))
a.index
d = df['created']
df['created'] = pd.to_datetime(df['created'], unit='s')
sort = df.sort_values(by='created')
series = df.set_index('created')

In [96]:

# a

DatetimeIndex(['2018-11-06 18:06:40', '2018-11-11 00:33:37',
               '2018-11-09 18:29:42', '2018-11-06 16:32:16',
               '2018-11-27 00:46:50', '2018-11-12 01:15:51',
               '2018-11-15 09:43:24', '2018-11-12 06:19:37',
               '2018-11-28 04:34:38', '2018-11-08 19:12:05',
               ...
               '2018-11-09 12:48:10', '2018-12-01 05:40:46',
               '2018-12-04 11:18:50', '2018-12-04 09:49:26',
               '2018-11-25 10:13:53', '2018-11-21 10:09:55',
               '2018-12-02 02:36:57', '2018-12-05 00:38:26',
               '2018-11-07 04:42:06', '2018-11-25 05:14:05'],
              dtype='datetime64[ns]', name='created', length=277, freq=None)

In [63]:
import matplotlib.pyplot as plt
ts = Series(df['created'])
ts

NameError: name 'Series' is not defined

In [37]:
import numpy as np

dt = pd.to_datetime(df['created'])


0     1970-01-01 00:00:01.541527600
1     1970-01-01 00:00:01.541896417
2     1970-01-01 00:00:01.541788182
3     1970-01-01 00:00:01.541521936
4     1970-01-01 00:00:01.543279610
5     1970-01-01 00:00:01.541985351
6     1970-01-01 00:00:01.542275004
7     1970-01-01 00:00:01.542003577
8     1970-01-01 00:00:01.543379678
9     1970-01-01 00:00:01.541704325
10    1970-01-01 00:00:01.542703293
11    1970-01-01 00:00:01.542163382
12    1970-01-01 00:00:01.543385451
13    1970-01-01 00:00:01.542327486
14    1970-01-01 00:00:01.543342996
15    1970-01-01 00:00:01.542855142
16    1970-01-01 00:00:01.541795760
17    1970-01-01 00:00:01.542926561
18    1970-01-01 00:00:01.543446622
19    1970-01-01 00:00:01.543630341
20    1970-01-01 00:00:01.543483721
21    1970-01-01 00:00:01.542373454
22    1970-01-01 00:00:01.543605384
23    1970-01-01 00:00:01.543946537
24    1970-01-01 00:00:01.542114047
25    1970-01-01 00:00:01.541552575
26    1970-01-01 00:00:01.542825441
27    1970-01-01 00:00:01.54

In [20]:
comment_data['created']

0       1970-01-01 00:00:01.541504467
1       1970-01-01 00:00:01.541506804
2       1970-01-01 00:00:01.541518260
3       1970-01-01 00:00:01.541523104
4       1970-01-01 00:00:01.541873642
5       1970-01-01 00:00:01.541903999
6       1970-01-01 00:00:01.541868747
7       1970-01-01 00:00:01.541949178
8       1970-01-01 00:00:01.541952652
9       1970-01-01 00:00:01.542052633
10      1970-01-01 00:00:01.542158019
11      1970-01-01 00:00:01.542219048
12      1970-01-01 00:00:01.541904328
13      1970-01-01 00:00:01.541879001
14      1970-01-01 00:00:01.541867767
15      1970-01-01 00:00:01.541869154
16      1970-01-01 00:00:01.541868038
17      1970-01-01 00:00:01.541868068
18      1970-01-01 00:00:01.541868353
19      1970-01-01 00:00:01.541885841
20      1970-01-01 00:00:01.541952574
21      1970-01-01 00:00:01.541885743
22      1970-01-01 00:00:01.541891938
23      1970-01-01 00:00:01.541919580
24      1970-01-01 00:00:01.541931118
25      1970-01-01 00:00:01.541874496
26      1970

In [18]:
comment_data['created'] = 

0       1970-01-01 00:00:01.541504467
1       1970-01-01 00:00:01.541506804
2       1970-01-01 00:00:01.541518260
3       1970-01-01 00:00:01.541523104
4       1970-01-01 00:00:01.541873642
5       1970-01-01 00:00:01.541903999
6       1970-01-01 00:00:01.541868747
7       1970-01-01 00:00:01.541949178
8       1970-01-01 00:00:01.541952652
9       1970-01-01 00:00:01.542052633
10      1970-01-01 00:00:01.542158019
11      1970-01-01 00:00:01.542219048
12      1970-01-01 00:00:01.541904328
13      1970-01-01 00:00:01.541879001
14      1970-01-01 00:00:01.541867767
15      1970-01-01 00:00:01.541869154
16      1970-01-01 00:00:01.541868038
17      1970-01-01 00:00:01.541868068
18      1970-01-01 00:00:01.541868353
19      1970-01-01 00:00:01.541885841
20      1970-01-01 00:00:01.541952574
21      1970-01-01 00:00:01.541885743
22      1970-01-01 00:00:01.541891938
23      1970-01-01 00:00:01.541919580
24      1970-01-01 00:00:01.541931118
25      1970-01-01 00:00:01.541874496
26      1970

In [13]:
from sqlalchemy import create_engine
conn_string = 'mysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = sql_cred["host"], 
    user = sql_cred["user"],
    password = sql_cred["password"], 
    db = 'MemeNews')
engine = create_engine(conn_string)

comment_data.to_sql('every_comment', index=True, con = engine, if_exists='append')
# comment_data.to_sql('all_comments', index=True, con = engine, if_exists='append')
# engine.execute('select * from articles').fetchall()

In [ ]:
#  comment_dict = { "post_id":[], 
#                   'post_title':[],

#                 "id": [],
#                         "author":[], 
#                         "body":[],
#                         "created": [],
#                          'score':[],
#                  'is_submitter':[],
#                   'parent_id':[]}
                         
                         
# for submission in reddit.subreddit('news').controversial(time_filter = 'year', limit=10000):
#     submission.comment_sort = 'top'
# #     for comment in submission.comments.list():
#     submission.comments.replace_more(limit=0)

            
    
    
#     for top_level_comment in submission.comments:
#         comment_dict['post_id'].append(submission.id)
#         comment_dict['is_submitter'].append(top_level_comment.is_submitter)
#         comment_dict['id'].append(top_level_comment.id)
#         comment_dict['author'].append(top_level_comment.author)
#         comment_dict['body'].append(re.sub(r'[^\x00-\x7F]', '', top_level_comment.body))
#         comment_dict['score'].append(top_level_comment.score)
#         comment_dict['created'].append(top_level_comment.created_utc)
#         comment_dict['parent_id'].append('None')
#         comment_dict['post_title'].append(submission.title)

#         for second_level_comment in top_level_comment.replies:
#             comment_dict['post_id'].append(submission.id)
#             comment_dict['id'].append(second_level_comment.id)
#             comment_dict['body'].append(re.sub(r'[^\x00-\x7F]', '',second_level_comment.body))
#             comment_dict['score'].append(second_level_comment.score)
#             comment_dict['is_submitter'].append(second_level_comment.is_submitter)
#             comment_dict['parent_id'].append(top_level_comment.id)
#             comment_dict['created'].append(second_level_comment.created_utc)
#             comment_dict['post_title'].append(submission.title)
#             try:
#                 comment_dict['author'].append(second_level_comment.author.name)
#             except:
#                 comment_dict['author'].append('None')


# #             print(comment_dict)
#     comment_data = pd.DataFrame(comment_dict)
# #         prin
# comment_data

In [ ]:
comment_data['post_id'][0]

In [ ]:
for i, row in df.iterrows():
    row['body']

In [ ]:

# comment_data.to_sql('top_comments', index=True, con = engine, if_exists='append')

In [ ]:
comment_data = comment_data.drop_duplicates('body')
df = comment_data[comment_data['body']!='None']
df = df[df['body']!='[removed]']
df

In [ ]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer

In [ ]:
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%"]
stop_words = text.ENGLISH_STOP_WORDS.union(punc)
bodies = df['body'].values
# parent_bodies = parent['parent-body'].values
# vectorizer = TfidfVectorizer(stop_words = stop_words)
# X = vectorizer.fit_transform(bodies)


In [ ]:
word_features = vectorizer.get_feature_names()
# word_features


In [ ]:
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def tokenize(text):
    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
vectorizer2 = TfidfVectorizer(stop_words = stop_words, tokenizer = tokenize)
X2 = vectorizer2.fit_transform(bodies)
word_features2 = vectorizer2.get_feature_names()
word_features2[:50]

In [ ]:
vectorizer3 = TfidfVectorizer(stop_words = stop_words, tokenizer = tokenize, max_features = 1000)
X3 = vectorizer3.fit_transform(bodies)
words = vectorizer3.get_feature_names()

In [ ]:
kmeans = KMeans(n_clusters = 5, n_init = 5, n_jobs = -1)
kmeans.fit(X3)

In [ ]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

In [ ]:
df['cluster'] = kmeans.labels_
clusters = df.groupby(['cluster']).size()
clusters

In [ ]:

df[df['cluster'] == 0]['body']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

clusters = df.groupby(['cluster', 'post_title']).size()
fig2, ax2 = plt.subplots(figsize = (30, 15))
sns.heatmap(clusters.unstack(level = 'post_title'), ax = ax2, cmap = 'Reds')

ax2.set_xlabel('variety', fontdict = {'weight': 'bold', 'size': 24})
ax2.set_ylabel('cluster', fontdict = {'weight': 'bold', 'size': 24})
for label in ax2.get_xticklabels():
    label.set_size(16)
    label.set_weight("bold")
for label in ax2.get_yticklabels():
    label.set_size(16)
    label.set_weight("bold")